# Installation and Setup

In [21]:
%%capture
#install aimodelshare library
!pip install transformers
!pip install -q tf-models-official
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

In [23]:
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [24]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
import os
import shutil


# Read Data



In [9]:
## Google Colab- Google Drive connections instructions 

# This step needs to be done for the first time when you're reading something from the Shared Project Folder 
# Please navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "NLP-Group1-FinalProj" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project drive folder through your own drive.


# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [15]:
groc_data = pd.read_csv('drive/My Drive/NLP-Group1-FinalProj/Data/Processed/groc_amz_data_processed.csv.gz', sep=',', compression="gzip")
groc_data = groc_data.drop(columns=groc_data.columns[0])

# Data Preparation

In [25]:
groc_data_trial = groc_data.head(10000)
groc_data_trial = groc_data_trial['review_text_processed'].squeeze()

In [26]:
groc_data_trial

0       no sugar, gmo garbage, fillers come store boug...
1       this absolute, undisputed favorite tea right n...
2       i ordered spongbob slippers i got john cena ha...
3       the cart fine works purpose i bought it. (farm...
4       this product archer farms best drink mix ever....
                              ...                        
9995    this sauce tastes great ...its spicy made gree...
9996    happy able taste fine product...it rich taste ...
9997    love item. bought mexico 98 cents. ad said 5 o...
9998    i love sauce. this sauce made showcase guajill...
9999    this sauce best ever it tastes freaking delici...
Name: review_text_processed, Length: 10000, dtype: object

# Laod Model and Prediction

In [27]:
# Path for the saved model in GDrive
saved_model_path = 'drive/My Drive/NLP-Group1-FinalProj/Models/Bert_IMDB_tuned_model2'

In [28]:
bert_cm2_classifier = tf.saved_model.load(saved_model_path)

##Sample check

In [29]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

In [31]:
groc_data_trial_sample = groc_data_trial[:10]

In [33]:
reloaded_results = tf.sigmoid(bert_cm2_classifier(tf.constant(groc_data_trial_sample)))

print('Results from the saved model:')
print_my_examples(groc_data_trial_sample, reloaded_results)

Results from the saved model:
input: no sugar, gmo garbage, fillers come store bought extracts. this stuff amazing. i use everything baking cooking even suggested coffee saying lot i normally care flavored coffee! you cannot go wrong this. i've ordered merchant before, customer satisfaction priority service quick, shipped right tracking even! i'll buying gls goods again! i use vanilla! : score: 0.998977
input: this absolute, undisputed favorite tea right now. i love darjeeling, i'm wildly fond lighter, first flush ones delicate. this darjeeling, especially steeped while, good tannic bite. it's bright warm time, pretty much explodes classic 'darjeeling' flavor. it's even remotely delicate, neither hard-edged. it's sort like good-looking men bespoke suits -- strong refined. i use boiling water, steep 4-5 minutes, large mug use one splenda tiny splash milk. then get way, tea, i take world! : score: 0.999460
input: i ordered spongbob slippers i got john cena happy ... son looking forward s